In [1]:
import os, re
import pandas as pd
import numpy as np

# #显示所有列
# pd.set_option('display.max_columns', None)

# #显示所有行
# pd.set_option('display.max_rows', None)

pd.set_option('max_colwidth',100) 

In [2]:
folders = [
    # r'D:\code\forecast_model\notebook\20250811_base',
    # r'D:\code\forecast_model\notebook\20250811_small_large',
    # r'D:\code\forecast_model\notebook\20250811_extraLarge',
    # r"D:\code\forecast_model\notebook\20250815_base_large",
    # r'D:\code\forecast_model\notebook\20250817_base',
    # r'D:\code\forecast_model\notebook\20250820_base',
    # r'D:\code\forecast_model\notebook\20250820_bigger',
    # r'D:\code\forecast_model\notebook\20250821_mlp_base2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline3',
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline2",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline3",
    # r'D:\code\forecast_model\notebook\20250825_tcn_baseline',
    r"D:\code\forecast_model\notebook\20250826_tcn_filt_days",
    ]

In [3]:
data = pd.DataFrame()
for folder in folders:
    for i in os.listdir(folder):
        if 'muilt_model' == i:continue
        # 20250811_base_P100_DeepLOB_v2_input_indepent	0.558035	2.77h
        # 20250811_base_P100_DeepLOB_v2_overfit
        if 'input_indepent' in i or 'overfit' in i: continue

        if i.endswith('.csv'): continue
        if i.endswith('.xlsx'): continue

        code = i.split('_')[3]
        fold = i.split('_')[4]

        train_folder = os.path.join(folder, i)
        if not os.path.isdir(train_folder): continue

        result_file = os.path.join(train_folder, 'result.csv')
        if not os.path.exists(result_file): continue

        # 修复替换
        _data = open(result_file, 'r').readlines()
        _data[1] = _data[1].replace('(30, 2, 1)', '30@2@1')
        with open(result_file, 'w') as f:
            f.writelines(_data)

        _data = pd.read_csv(result_file)
        # _data['describe.1'] = _data['describe.1'] + "_" + code + f"_fold{fold}"

        # 判断是否有nan
        if _data.isnull().values.any(): 
            print(i)
            continue

        if 'test_class_f1_0' in list(_data):
            _data[f'test_mean_class_f1'] = (_data[f'test_class_f1_0'] + _data[f'test_class_f1_1']) / 2
        elif 'test_final_class_f1_0' in list(_data):
            # 同时采用 best/final
            d2 = _data.copy()
            d2[f'test_mean_class_f1'] = _data[f'test_best_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                d2[f'test_mean_class_f1'] += _data[f'test_best_class_f1_{i}']
            d2[f'test_mean_class_f1'] = d2[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            d2['train_title'] += '_best'

            _data[f'test_mean_class_f1'] = _data[f'test_final_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                _data[f'test_mean_class_f1'] += _data[f'test_final_class_f1_{i}']
            _data[f'test_mean_class_f1'] = _data[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            _data['train_title'] += '_final'

            _data = pd.concat([_data, d2], ignore_index=True)

        # for _type in ['train', 'val', 'test_best', 'test_final', 'test_dummy']:
        #     _data[f'{_type}_mean_class_f1'] = (_data[f'{_type}_class_f1_0'] + _data[f'{_type}_class_f1_1']) / 2

        # for _type in ['train', 'val', 'test_best', 'test_final']:
            # _data[f'{_type}_mean_class_f1_enhanced_pct'] = 100 * (_data[f'{_type}_mean_class_f1'] - _data['test_dummy_mean_class_f1']) / _data['test_dummy_mean_class_f1']

        data = pd.concat([data, _data], ignore_index=True)

try:
    data['mean_test_01_f1'] = (data['test_best_class_f1_0'] + data['test_best_class_f1_1']) / 2
    data['mean_val_01_f1'] = (data['val_class_f1_0'] + data['val_class_f1_1']) / 2
except:
    pass

# 只使用 top 数据集 predict_n100
# data = data.loc[data['train_title'].str.contains('top5')].reset_index(drop=True)
# data = data.loc[data['train_title'].str.contains('predict_n100')].reset_index(drop=True)

data

,train_title,root,alist_upload_folder,amp,epochs,batch_n,batch_size,learning_rate,abs_learning_rate,no_better_stop,...,test_dummy_mean_class_f1,train_mean_class_f1_enhanced_pct,val_mean_class_f1_enhanced_pct,test_best_mean_class_f1_enhanced_pct,test_final_mean_class_f1_enhanced_pct,each_epoch_cost,cost,test_mean_class_f1,mean_test_01_f1,mean_val_01_f1
0,20250826_filt_P100_200_seed0_IDX0_final,20250826_filt_P100_200_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.498663,100.536225,69.813663,84.504811,64.342986,0.02h,3.72h,0.817302,0.920057,0.846797
1,20250826_filt_P100_200_seed0_IDX0_best,20250826_filt_P100_200_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.498663,100.536225,69.813663,84.504811,64.342986,0.02h,3.72h,0.918732,0.920057,0.846797
2,20250826_filt_P100_200_seed1_IDX3_final,20250826_filt_P100_200_seed1_IDX3,train_data,no,200,128,8192,0.0384,0,0,...,0.500067,99.973265,69.428144,69.049878,63.761490,0.02h,4.17h,0.816619,0.845363,0.847254
3,20250826_filt_P100_200_seed1_IDX3_best,20250826_filt_P100_200_seed1_IDX3,train_data,no,200,128,8192,0.0384,0,0,...,0.500067,99.973265,69.428144,69.049878,63.761490,0.02h,4.17h,0.844921,0.845363,0.847254
4,20250826_filt_P100_200_seed2_IDX6_final,20250826_filt_P100_200_seed2_IDX6,train_data,no,200,128,8192,0.0384,0,0,...,0.502980,98.815124,69.150256,81.304272,61.693208,0.02h,4.18h,0.811401,0.911924,0.850791
5,20250826_filt_P100_200_seed2_IDX6_best,20250826_filt_P100_200_seed2_IDX6,train_data,no,200,128,8192,0.0384,0,0,...,0.502980,98.815124,69.150256,81.304272,61.693208,0.02h,4.18h,0.908133,0.911924,0.850791
6,20250826_filt_P100_200_seed3_IDX9_final,20250826_filt_P100_200_seed3_IDX9,train_data,no,200,128,8192,0.0384,0,0,...,0.501496,99.403532,67.658107,78.680748,61.078987,0.02h,4.21h,0.805011,0.896076,0.840798
7,20250826_filt_P100_200_seed3_IDX9_best,20250826_filt_P100_200_seed3_IDX9,train_data,no,200,128,8192,0.0384,0,0,...,0.501496,99.403532,67.658107,78.680748,61.078987,0.02h,4.21h,0.891392,0.896076,0.840798
8,20250826_filt_P100_200_seed4_IDX12_final,20250826_filt_P100_200_seed4_IDX12,train_data,no,200,128,8192,0.0384,0,0,...,0.497787,100.889094,70.061792,79.382916,61.847477,0.02h,4.17h,0.802639,0.892945,0.846546
9,20250826_filt_P100_200_seed4_IDX12_best,20250826_filt_P100_200_seed4_IDX12,train_data,no,200,128,8192,0.0384,0,0,...,0.497787,100.889094,70.061792,79.382916,61.847477,0.02h,4.17h,0.887710,0.892945,0.846546


In [4]:
if len(folders) == 1:
    data.to_csv(os.path.join(folders[0], 'result.csv'), index=False)


In [5]:
list(data)

['train_title',
 'root',
 'alist_upload_folder',
 'amp',
 'epochs',
 'batch_n',
 'batch_size',
 'learning_rate',
 'abs_learning_rate',
 'no_better_stop',
 'checkpointing_steps',
 'label_smoothing',
 'weight_decay',
 'classify',
 'y_n',
 'debug',
 'test',
 'seed',
 'label_train',
 'label_val',
 'label_test_final',
 'model',
 'grad_norm_best',
 'grad_norm',
 'total_grad_norm_best',
 'total_grad_norm',
 'train_loss_best',
 'train_loss',
 'train_acc_best',
 'train_acc',
 'train_f1_best',
 'train_f1',
 'train_recall_best',
 'train_recall',
 'train_class_acc_0_best',
 'train_class_acc_0',
 'train_class_mcc_0_best',
 'train_class_mcc_0',
 'train_class_f1_0_best',
 'train_class_f1_0',
 'train_class_recall_0_best',
 'train_class_recall_0',
 'train_class_acc_1_best',
 'train_class_acc_1',
 'train_class_mcc_1_best',
 'train_class_mcc_1',
 'train_class_f1_1_best',
 'train_class_f1_1',
 'train_class_recall_1_best',
 'train_class_recall_1',
 'val_loss_best',
 'val_loss',
 'val_acc_best',
 'val_acc',

In [6]:
data[['train_title', 'label_train']]

,train_title,label_train
0,20250826_filt_P100_200_seed0_IDX0_final,1(9397)@1(9397)
1,20250826_filt_P100_200_seed0_IDX0_best,1(9397)@1(9397)
2,20250826_filt_P100_200_seed1_IDX3_final,1(9397)@1(9397)
3,20250826_filt_P100_200_seed1_IDX3_best,1(9397)@1(9397)
4,20250826_filt_P100_200_seed2_IDX6_final,1(9397)@1(9397)
5,20250826_filt_P100_200_seed2_IDX6_best,1(9397)@1(9397)
6,20250826_filt_P100_200_seed3_IDX9_final,1(9397)@1(9397)
7,20250826_filt_P100_200_seed3_IDX9_best,1(9397)@1(9397)
8,20250826_filt_P100_200_seed4_IDX12_final,1(9397)@1(9397)
9,20250826_filt_P100_200_seed4_IDX12_best,1(9397)@1(9397)


In [7]:
# 多seed平均
key_col_name = ['train_f1']
key_col_name = ['train_loss', 'train_f1', 'val_f1', 'test_final_f1']

d = data.loc[:, key_col_name + ['cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'seed' not in i]))

# 只考察 final
_match_bool = data['train_title'].apply(lambda x: 'final' in x)
d = d.loc[_match_bool]

# # 根据 idx 区分类别
# # 用于未区分title的情况
# _cls = d['train_title'].apply(lambda x: [(int(i.replace('IDX', '')) % 3) for i in x.split('_') if 'IDX' in i][0])
# _cls = _cls.apply(lambda x: ['nomove30', 'nomove50', 'nomove70'][x])
# d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i])) + '_' + _cls

# title 已经区分
d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values('train_f1',ascending=False)


,train_loss,train_f1,val_f1,test_final_f1,cost
train_title,,,,,
20250826_filt_P100_200_final,0.000014,1.0,0.846448,0.813110,4.09h
20250826_filt_P100_250_final,0.000007,1.0,0.750465,0.758146,4.2620000000000005h
20250826_filt_P100_300_final,0.000006,1.0,0.635058,0.394024,4.35h


In [8]:
# 多IDX平均
key_col_name = 'val_f1'
key_col_name = 'label_train'

d = data.loc[:, [key_col_name, 'cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i and 'seed' not in i ]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values([key_col_name],ascending=False)

TypeError: agg function failed [how->mean,dtype->object]